## PREPARACIO DELS DATASETS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from statistics import mean, median
import os

In [2]:
def x(alpha, delta, alpha_c, delta_c):
    x = np.cos(np.radians(delta))*np.sin(np.radians(alpha-alpha_c))
    return np.degrees(x)

def y(alpha, delta, alpha_c, delta_c):
    y = np.sin(np.radians(delta))*np.cos(np.radians(delta_c))-np.cos(np.radians(delta))*np.sin(np.radians(delta_c))*np.cos(np.radians(alpha-alpha_c))
    return np.degrees(y)

def mu_x(alpha, delta, mu_alpha, mu_delta, alpha_c, delta_c):
    mu_x = mu_alpha*np.cos(np.radians(alpha-alpha_c)) - mu_delta*np.sin(np.radians(delta))*np.sin(np.radians(alpha-alpha_c))
    return mu_x

def mu_y(alpha, delta, mu_alpha, mu_delta, alpha_c, delta_c):
    mu_y = mu_alpha*np.sin(np.radians(delta_c))*np.sin(np.radians(alpha-alpha_c)) + mu_delta*(np.cos(np.radians(delta))*np.cos(np.radians(delta_c))+np.sin(np.radians(delta))*np.sin(np.radians(delta_c))*np.cos(np.radians(alpha-alpha_c)))
    return mu_y

#### DADES GAIA

In [3]:
new = 'E:\DOWNLOADS\TFG\GAIA2'
os.chdir(new)

In [4]:
GAIA = pd.read_csv('LMC_eDR3_TFG_Arnau-result.csv')

Afegim  x, y, mu_x, mu_y, i bp-rp

In [5]:
alpha_c_LMC = 81.28 #degrees
delta_c_LMC = -69.78 #degrees

GAIA['x'] = x(GAIA['ra'],GAIA['dec'],alpha_c_LMC,delta_c_LMC)
GAIA['y'] = y(GAIA['ra'],GAIA['dec'],alpha_c_LMC,delta_c_LMC)
GAIA['mu_x'] = mu_x(GAIA['ra'],GAIA['dec'],GAIA['pmra'],GAIA['pmdec'],alpha_c_LMC,delta_c_LMC)
GAIA['mu_y'] = mu_y(GAIA['ra'],GAIA['dec'],GAIA['pmra'],GAIA['pmdec'],alpha_c_LMC,delta_c_LMC)
GAIA['BP_RP'] = GAIA['phot_bp_mean_mag'] - GAIA['phot_rp_mean_mag']

In [6]:
print(len(GAIA))

17388570


Hi ha NaN en 'phot_bp_mean_mag' i 'phot_bp_mean_mag' en algunes estrelles

In [7]:
GAIA = GAIA.dropna(subset=['BP_RP']) 

In [8]:
print(len(GAIA))

15501760


In [9]:
new = 'E:\TFG\Dades_Gaia'
os.chdir(new)

In [10]:
GAIA.to_csv('GAIA.csv')

In [11]:
GAIA = None #Alliberem memoria

#### DADES SIMULACIO GOG

In [12]:
new = 'E:\DOWNLOADS\TFG\Simulacio'
os.chdir(new)

In [13]:
MWs = pd.read_csv('MW_s.csv') #no hem canviat les dades de MW
LMCs = pd.read_csv('LMC_s2.csv')

In [14]:
alpha_c_LMC = 81.28 #degrees
delta_c_LMC = -69.78 #degrees

LMCs['ra'] = np.degrees(LMCs['Alpha'])
LMCs['dec'] = np.degrees(LMCs['Delta'])

MWs['ra'] = np.degrees(MWs['Alpha'])
MWs['dec'] = np.degrees(MWs['Delta'])

LMCs['x'] = x(LMCs['ra'],LMCs['dec'],alpha_c_LMC,delta_c_LMC)
LMCs['y'] = y(LMCs['ra'],LMCs['dec'],alpha_c_LMC,delta_c_LMC)
LMCs['mu_x'] = mu_x(LMCs['ra'],LMCs['dec'],LMCs['MuAlphaStar'],LMCs['MuDelta'],alpha_c_LMC,delta_c_LMC)
LMCs['mu_y'] = mu_y(LMCs['ra'],LMCs['dec'],LMCs['MuAlphaStar'],LMCs['MuDelta'],alpha_c_LMC,delta_c_LMC)

MWs['x'] = x(MWs['ra'],MWs['dec'],alpha_c_LMC,delta_c_LMC)
MWs['y'] = y(MWs['ra'],MWs['dec'],alpha_c_LMC,delta_c_LMC)
MWs['mu_x'] = mu_x(MWs['ra'],MWs['dec'],MWs['MuAlphaStar'],MWs['MuDelta'],alpha_c_LMC,delta_c_LMC)
MWs['mu_y'] = mu_y(MWs['ra'],MWs['dec'],MWs['MuAlphaStar'],MWs['MuDelta'],alpha_c_LMC,delta_c_LMC)

Distribució de Magnituds \
En la simulacio tenim les dades del flux així que utilitzem la formula:\
\
$G = G_0 - 2.5 log Flux$ \
\
Amb: \
${G}_0 = 25.620$ \
${BP}_0 = 25.264$ \
${RP}_0 = 24.510$ 

In [15]:
G0 = 25.620 
BP0 = 25.264 
𝑅𝑃0 = 24.510
for i in (LMCs, MWs):
    i['BP_RP'] = (BP0 - 2.5*np.log10(i.BpMean_FluxMean)) - (RP0 - 2.5*np.log10(i.RpMean_FluxMean))
    i['phot_g_mean_mag'] = G0 - 2.5*np.log10(i.GMean_FluxMean)

In [16]:
MWs['Type'] = 0
LMCs['Type'] = 1
LMC_Rs = pd.concat([LMCs, MWs], ignore_index=True)

In [17]:
print(len(LMC_Rs))
LMC_Rs.dropna()
print(len(LMC_Rs))

1201329
1201329


In [18]:
new = 'E:\TFG\Dades_Simulació'
os.chdir(new)

In [19]:
LMC_Rs.to_csv('GOG_Simulation.csv')

In [20]:
MWs = None
LMCs = None
LMC_Rs = None

#### DADES SH

In [3]:
new = 'E:\DOWNLOADS\TFG\SH'
os.chdir(new)

In [4]:
MWsh = pd.read_csv('SH_MW.csv')

In [5]:
LMCsh = pd.read_csv('SH_LMC.csv')

In [6]:
MWsh.columns

Index(['Unnamed: 0', 'ra', 'dec', 'source_id', 'dist05', 'dist16', 'dist50',
       'dist84', 'dist95', 'av05', 'av16', 'av50', 'av84', 'av95', 'teff16',
       'teff50', 'teff84', 'logg16', 'logg50', 'logg84', 'met16', 'met50',
       'met84', 'mass16', 'mass50', 'mass84', 'ag50', 'abp50', 'arp50',
       'bprp0', 'mg0', 'xgal', 'ygal', 'zgal', 'rgal', 'fidelity',
       'bp_rp_excess_corr', 'sh_photoflag', 'sh_outflag', 'x', 'y'],
      dtype='object')

In [7]:
len(MWsh)

1582565

In [8]:
len(LMCsh)

1041430

In [9]:
alpha_c_LMC = 81.28 #degrees
delta_c_LMC = -69.78 #degrees

for i in [LMCsh,MWsh]:
    i['x'] = x(i['ra'],i['dec'],alpha_c_LMC,delta_c_LMC)
    i['y'] = y(i['ra'],i['dec'],alpha_c_LMC,delta_c_LMC)
    #i['mu_x'] = mu_x(i['ra'],i['dec'],i['pmra'],i['pmdec'],alpha_c_LMC,delta_c_LMC)
    #i['mu_y'] = mu_y(i['ra'],i['dec'],i['pmra'],i['pmdec'],alpha_c_LMC,delta_c_LMC)

LMCsh['BP_RP'] = LMCsh.bprp0
MWsh['BP_RP'] = MWsh.bprp0
LMCsh['phot_g_mean_mag'] = LMCsh.mg0
MWsh['phot_g_mean_mag'] = MWsh.mg0

LMCsh = LMCsh.dropna()
MWsh = MWsh.dropna()

In [10]:
new = 'E:\TFG\Dades_SH'
os.chdir(new)

In [11]:
MWsh.to_csv('SH_MW.csv')
LMCsh.to_csv('SH_LMC.csv')

#### DADES RR Lyrae

In [ ]:
new = 'E:\DOWNLOADS\TFG\CEFEIDES'
os.chdir(new)

In [ ]:
LMCcef = pd.read_csv('Cep_LMC.csv')

In [ ]:
alpha_c_LMC = 81.28 #degrees
delta_c_LMC = -69.78 #degrees

LMCcef['x'] = x(LMCcef['ra'],LMCcef['dec'],alpha_c_LMC,delta_c_LMC)
LMCcef['y'] = y(LMCcef['ra'],LMCcef['dec'],alpha_c_LMC,delta_c_LMC)
#i['mu_x'] = mu_x(i['ra'],i['dec'],i['pmra'],i['pmdec'],alpha_c_LMC,delta_c_LMC)
#i['mu_y'] = mu_y(i['ra'],i['dec'],i['pmra'],i['pmdec'],alpha_c_LMC,delta_c_LMC)
LMCcef['BP_RP'] = LMCcef.bp_rp
print(len(LMCcef))

In [ ]:
new = 'E:\TFG\Dades_Cepheids'
os.chdir(new)

In [ ]:
LMCcef.to_csv('Cef_LMC.csv')

#### DADES RR Lyrae

In [ ]:
new = 'E:\DOWNLOADS\TFG\RR'
os.chdir(new)

In [ ]:
LMC_RR = pd.read_csv('RR_LMC_EDR3.csv')

In [ ]:
alpha_c_LMC = 81.28 #degrees
delta_c_LMC = -69.78 #degrees

LMC_RR['x'] = x(LMC_RR['ra'],LMC_RR['dec'],alpha_c_LMC,delta_c_LMC)
LMC_RR['y'] = y(LMC_RR['ra'],LMC_RR['dec'],alpha_c_LMC,delta_c_LMC)
#i['mu_x'] = mu_x(i['ra'],i['dec'],i['pmra'],i['pmdec'],alpha_c_LMC,delta_c_LMC)
#i['mu_y'] = mu_y(i['ra'],i['dec'],i['pmra'],i['pmdec'],alpha_c_LMC,delta_c_LMC)
LMC_RR['BP_RP'] = LMC_RR.bp_rp
print(len(LMC_RR))

In [ ]:
new = 'E:\TFG\Dades_RR'
os.chdir(new)

In [ ]:
LMC_RR.to_csv('RR_LMC.csv')